<h1>Rotulação de Dados</h1>

Neste notebook é criado labels para os dados.

<h4>Rotulação de Tipo de Superfície</h4>

<table>
     <tr>
         <th>Valor</th>
         <th>Tipo</th>
         <th>Especificação</th>
     </tr>
     <tr>
         <td>0,1</td>
         <td>Sem Pavimento</td>
         <td>Terra</td>
     </tr>
     <tr>
         <td>0,1</td>
         <td>Pavimento Flexível</td>
         <td>Paralelepípedo/Hexagonal</td>
     </tr>
     <tr>
         <td>0,1</td>
         <td>Pavimento Rígido</td>
         <td>Asfalto</td>
     </tr>
</table>


<h4>Importação de Pacotes</h4>

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib notebook
pd.set_option("float_format", '{:0.10f}'.format)
pd.set_option('display.max_columns', 30)
%run "3 - Data Labeling.py"

<h4>Funções Utilitárias</h4>

In [ ]:
# Rotula cada amostra com base nos ranges de samples
def mapToLabel(sample, class_ranges):
    
    for interval in class_ranges:
        
        if(sample >= interval[0] and sample <= interval[1]):
            return 1
        
    return 0

# Adiciona uma classe de dado como coluna no dataset
def addClassToData(classes, class_name, class_ranges, data_size):
    class_values = pd.Series(range(0, data_size)).apply(mapToLabel, args=(class_ranges,))
    classes.insert(len(classes.columns), class_name, class_values, True)
    
# Método principal 
def processLabels(data_mpu, class_ranges):
    
    data_size = len(data_mpu)
    classes = pd.DataFrame()
    
    addClassToData(classes, 'land',        class_ranges['sem_pavimento'],      data_size)
    addClassToData(classes, 'cobblestone', class_ranges['pavimento_flexivel'], data_size)
    addClassToData(classes, 'asphalt',     class_ranges['pavimento_rigido'],   data_size)
    
    file = os.path.join(folder, 'dataset_labels.csv')
    classes.to_csv(file, index=False)
    print("Salvo em ", file)

    return classes

# Plot dados com classes
def plotClasses(data_mpu, classes):
    plt.figure(figsize=(16,6)) 
    data_mpu['gyro_y_below_suspension'].plot(color="g")
    data_mpu['acc_z_below_suspension'].plot(color="y")
    (data_mpu['speed'] * 3.6).plot()
    (classes['land'] * 20).plot(linewidth=4)
    (classes['cobblestone'] * 40).plot(linewidth=4)
    (classes['asphalt'] * 60).plot(linewidth=4)
    plt.legend()

<h4>Importação dos Dados</h4>

In [ ]:
dataset = workspace['datasets']['saveiro-1']
folder = os.path.join(workspace['settings']['folder'], dataset['folder'])
data_esquerda = pd.read_csv(os.path.join(folder, 'dataset_gps_mpu_left.csv'), float_precision="high")
data_direita = pd.read_csv(os.path.join(folder, 'dataset_gps_mpu_right.csv'), float_precision="high")

<h4>Rotulação de Dados</h4>

In [ ]:
classes = processLabels(data_esquerda.copy(), dataset.copy())

In [ ]:
plotClasses(data_esquerda, classes)

In [ ]:
plotClasses(data_direita, classes)